# Day 17: Trick Shot

In [1]:
example = """target area: x=20..30, y=-10..-5"""

In [2]:
def parse(input):
    """Parses targets from input."""
    x1, x2, y1, y2 = (int(value) for part in input.split(': ')[1].split(', ') for value in part.split('=')[1].split('..'))
    return set((x, y) for x in range(x1, 1 + x2) for y in range(y1, 1 + y2))
    
parse(example)

{(20, -10),
 (20, -9),
 (20, -8),
 (20, -7),
 (20, -6),
 (20, -5),
 (21, -10),
 (21, -9),
 (21, -8),
 (21, -7),
 (21, -6),
 (21, -5),
 (22, -10),
 (22, -9),
 (22, -8),
 (22, -7),
 (22, -6),
 (22, -5),
 (23, -10),
 (23, -9),
 (23, -8),
 (23, -7),
 (23, -6),
 (23, -5),
 (24, -10),
 (24, -9),
 (24, -8),
 (24, -7),
 (24, -6),
 (24, -5),
 (25, -10),
 (25, -9),
 (25, -8),
 (25, -7),
 (25, -6),
 (25, -5),
 (26, -10),
 (26, -9),
 (26, -8),
 (26, -7),
 (26, -6),
 (26, -5),
 (27, -10),
 (27, -9),
 (27, -8),
 (27, -7),
 (27, -6),
 (27, -5),
 (28, -10),
 (28, -9),
 (28, -8),
 (28, -7),
 (28, -6),
 (28, -5),
 (29, -10),
 (29, -9),
 (29, -8),
 (29, -7),
 (29, -6),
 (29, -5),
 (30, -10),
 (30, -9),
 (30, -8),
 (30, -7),
 (30, -6),
 (30, -5)}

In [3]:
def step(x, y, v_x, v_y):
    """Returns position and velocity after one step."""
    return x + v_x, y + v_y, v_x - 1 if v_x > 0 else 0, v_y - 1

step(0, 0, 1, 1)

(1, 1, 0, 0)

If a probe has an initial y-velocity >= 0, it will pass through y=0 on the way back down. At this point the probe will have a negative velocity with 1 greater magnitude than the intial velocity.

For example, a probe with initial y-velocity 3 will pass through y=0 with -4 y-velocity.

In [4]:
x, y, v_x, v_y = 0, 0, 0, 3
for s in range(1, 8):
    x, y, v_x, v_y = step(x, y, v_x, v_y)
    print(f'Step {s}: {(x, y)} {(v_x, v_y)}')

Step 1: (0, 3) (0, 2)
Step 2: (0, 5) (0, 1)
Step 3: (0, 6) (0, 0)
Step 4: (0, 6) (0, -1)
Step 5: (0, 5) (0, -2)
Step 6: (0, 3) (0, -3)
Step 7: (0, 0) (0, -4)


The initial y-velocity cannot be greater than the magnitude of the lowest target minus 1.

In [5]:
def max_v_y(targets):
    """Returns maximum initial y-velocity that can hit targets."""
    return abs(min(targets)[1]) - 1 

max_v_y(parse(example))

9

A probe's y velocities form an infinite arithmetic progression. We can consider a finite portion of this progression: from initial velocity until inflection (y-velocity = 0). 

A probe's highest y-position is the sum of the finite progression. 

A probe's highest position for an initial y-velocity 9 can be calculated as follows:

In [6]:
def max_height(v):
    """Returns max height of travelling with initial y-velocity v."""
    return int(v * (v + 1) / 2)

max_height(max_v_y(parse(example)))

45

Find the maximum height of a probe that will hit targets in input.

In [7]:
max_height(max_v_y(parse(open('day-17-input.txt').read())))

19503

# Part two

You can't start with an x-velocity greater than the rightmost target's x-position, otherwise the probe would to the right of the target area after the first step.

In [8]:
max(parse(example))[0]

30

You can't start with a y-velocity less than the lowest target's y-position, otherwise the probe would be lower than the target area after the first step.

In [9]:
min(parse(example))[1]

-10

A probe's final x-position must not be less than the leftmost position of the target area, otherwise the probe will never reach the target area horizontally.

x velocities form an [arithmetic progression](https://en.wikipedia.org/wiki/Arithmetic_progression): the difference between consecutive terms is constant. e.g., from initial velocity to final velocity: (5, 4, 3, 2, 1, 0).

The sum of a finite arithmetic progression is an arithmetic series. Here, this equates to the probe's final x-position.

The final x-position (series) for an intial x-velocity of 5 can be calculated as follows:

In [10]:
final_x = max_height

final_x(5)

15

We've got bounds for initial x-velocity and initial y-velocity.

Let's iterate through these and see what hits.

In [11]:
targets = parse(open('day-17-input.txt').read())
len(targets)

3060

In [12]:
import itertools

def hits(targets):
    """Yields initial velocities of probes that reach targets."""
    leftmost = min(targets)[0]
    rightmost = max(targets)[0]
    lowest = min(targets)[1]
    
    # Initial x velocities.
    v_xs = (v for v in range(rightmost + 1) if final_x(v) >= leftmost)
    # Map x position sequences to initial x velocities.
    x_pos = {
        # Sum terms in sequences from initial x velocity to 1. 
        v_x: [int(n * (v_x + v) / 2) for n, v in enumerate(range(v_x, 0, -1), start=1) if (n * (v_x + v) / 2) <= rightmost]
        for v_x in v_xs
    }
    
    # Initial y velocities.
    v_ys = range(lowest, max_v_y(targets) + 1)
    y_pos = {}
    for v_y in v_ys:
        # Determine the probe's final y position within target.
        # Take a shortcut by considering only movement below the y-axis.
        final_v_y = -(v_y + 1) if v_y > 0 else v_y
        final_y = final_v_y
        while (final_y + final_v_y - 1) >= lowest:
            final_v_y -= 1
            final_y += final_v_y

        # Then determine y positions for each step.
        y_pos[v_y] = [n * (v_y + v) / 2 for n, v in enumerate(range(v_y, final_v_y - 1, -1), start=1)]
    
    for v_x, xs in x_pos.items():
        for v_y, ys in y_pos.items():
            # Repeat the final x position forever if it has run out of steam.
            tweaked_xs = itertools.chain(xs, itertools.repeat(xs[-1])) if len(xs) > 1 and xs[-2] + 1 == xs[-1] else xs
            
            # If any positions of this trajectory are in targets, yield the initial velocities.
            if set(zip(tweaked_xs, ys)) & targets:
                yield v_x, v_y

len(set(hits(parse(example))))

112

Find number of starting velocities that hit target.

In [13]:
len(set(hits(parse(open('day-17-input.txt').read()))))

5200